<a href="https://colab.research.google.com/github/mariabandeira/AdministracaoPublica/blob/main/CadUnico/KNN/TrainningPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score
from imblearn.under_sampling import RandomUnderSampler

In [12]:
treated_data_RN = pd.read_csv('./BasesRN/cadunico_rn_cleaned.csv', sep=';', encoding='utf-8')

In [13]:
treated_data_RN.dropna(inplace=True)
treated_data_RN.drop_duplicates(inplace=True)

In [14]:
treated_data_RN.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167229 entries, 0 to 296061
Data columns (total 16 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   classf                      167229 non-null  int64  
 1   vlr_renda_media_fam         167229 non-null  float64
 2   cod_local_domic_fam         167229 non-null  float64
 3   qtd_comodos_domic_fam       167229 non-null  float64
 4   qtd_comodos_dormitorio_fam  167229 non-null  float64
 5   cod_material_piso_fam       167229 non-null  float64
 6   cod_material_domic_fam      167229 non-null  float64
 7   cod_agua_canalizada_fam     167229 non-null  float64
 8   cod_abaste_agua_domic_fam   167229 non-null  float64
 9   cod_banheiro_domic_fam      167229 non-null  float64
 10  cod_destino_lixo_domic_fam  167229 non-null  float64
 11  cod_iluminacao_domic_fam    167229 non-null  float64
 12  cod_calcamento_domic_fam    167229 non-null  float64
 13  ind_familia_quilomb

In [15]:
treated_data_RN.head()

,classf,vlr_renda_media_fam,cod_local_domic_fam,qtd_comodos_domic_fam,qtd_comodos_dormitorio_fam,cod_material_piso_fam,cod_material_domic_fam,cod_agua_canalizada_fam,cod_abaste_agua_domic_fam,cod_banheiro_domic_fam,cod_destino_lixo_domic_fam,cod_iluminacao_domic_fam,cod_calcamento_domic_fam,ind_familia_quilombola_fam,marc_pbf,qtde_pessoas
0,2,312.0,1.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,0,3
1,2,75.0,1.0,4.0,1.0,5.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,2.0,1,2
2,2,60.0,1.0,5.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1,5
3,2,394.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0,2
4,2,66.0,1.0,5.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1,3


In [8]:
list(treated_data_RN.columns)

[]

In [17]:
def classificar(x):
    if x == 1:
        return 1
    elif x == 2:
        return 2
    elif x == 3:
        return 3
    elif x == 4:
        return 4
    elif x == 5:
        return 5
    else:
        return 6

treated_data_RN['qtde_pessoas'] = treated_data_RN['qtde_pessoas'].apply(classificar)
treated_data_RN['qtde_pessoas'].value_counts()

,count
qtde_pessoas,
3,43484
2,43380
4,30465
1,24426
5,14775
6,10699


In [18]:
treated_data_RN.describe()

,classf,vlr_renda_media_fam,cod_local_domic_fam,qtd_comodos_domic_fam,qtd_comodos_dormitorio_fam,cod_material_piso_fam,cod_material_domic_fam,cod_agua_canalizada_fam,cod_abaste_agua_domic_fam,cod_banheiro_domic_fam,cod_destino_lixo_domic_fam,cod_iluminacao_domic_fam,cod_calcamento_domic_fam,ind_familia_quilombola_fam,marc_pbf,qtde_pessoas
count,167229.000000,167229.000000,167229.000000,167229.000000,167229.000000,167229.000000,167229.000000,167229.000000,167229.000000,167229.000000,167229.000000,167229.000000,167229.000000,167229.00000,167229.000000,167229.000000
mean,2.416650,206.979813,1.327987,4.763767,1.798570,2.996693,1.277894,1.192939,1.604542,1.046158,1.518247,1.308810,1.997596,1.99466,0.552004,2.999282
std,0.774275,270.894807,0.469482,1.487441,0.747932,1.499163,0.784183,0.394607,1.048988,0.209828,0.918301,0.994599,0.967229,0.07288,0.497290,1.398233
min,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,0.000000,1.000000
25%,2.000000,37.000000,1.000000,4.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.00000,0.000000,2.000000
50%,3.000000,91.000000,1.000000,5.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.00000,1.000000,3.000000
75%,3.000000,276.000000,2.000000,6.000000,2.000000,5.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,3.000000,2.00000,1.000000,4.000000
max,3.000000,2862.000000,2.000000,20.000000,20.000000,7.000000,8.000000,2.000000,4.000000,2.000000,6.000000,6.000000,3.000000,2.00000,1.000000,6.000000


In [19]:
treated_data_RN.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167229 entries, 0 to 296061
Data columns (total 16 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   classf                      167229 non-null  int64  
 1   vlr_renda_media_fam         167229 non-null  float64
 2   cod_local_domic_fam         167229 non-null  float64
 3   qtd_comodos_domic_fam       167229 non-null  float64
 4   qtd_comodos_dormitorio_fam  167229 non-null  float64
 5   cod_material_piso_fam       167229 non-null  float64
 6   cod_material_domic_fam      167229 non-null  float64
 7   cod_agua_canalizada_fam     167229 non-null  float64
 8   cod_abaste_agua_domic_fam   167229 non-null  float64
 9   cod_banheiro_domic_fam      167229 non-null  float64
 10  cod_destino_lixo_domic_fam  167229 non-null  float64
 11  cod_iluminacao_domic_fam    167229 non-null  float64
 12  cod_calcamento_domic_fam    167229 non-null  float64
 13  ind_familia_quilomb

### Treinando o modelo KNN

In [20]:
# analisando a distribuição da variável alvo (marc_pbf)
target_distribution = treated_data_RN['marc_pbf'].value_counts(normalize=True) * 100

print(round(target_distribution, 2))

marc_pbf
1    55.2
0    44.8
Name: proportion, dtype: float64


In [21]:
# separar X (features) e y (target)
X = treated_data_RN.drop('marc_pbf', axis=1)
y = treated_data_RN['marc_pbf']

# treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# criando uma instância do RandomUnderSampling
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')

# balanceando os dados
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(y_resampled.value_counts())

marc_pbf
0    52437
1    52437
Name: count, dtype: int64


In [22]:
# aplicar padronização nas features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

print("Primeiros dados escalados (padronizados):")
print(X_train_scaled[:5])

Primeiros dados escalados (padronizados):
[[ 0.75975286  2.00842718 -0.68930805 -1.19387926 -1.07005825 -0.67888863
  -0.34944544 -0.48122804 -0.56836639 -0.21670295 -0.55755989 -0.30495667
   1.05395763  0.07194224 -1.41686922]
 [-0.52785273  0.78871059 -0.68930805 -1.86306982 -1.07005825 -0.67888863
   0.94658104 -0.48122804 -0.56836639 -0.21670295 -0.55755989 -0.30495667
  -1.01402715  0.07194224 -0.69649989]
 [ 0.75975286  0.86694314 -0.68930805  0.14450187 -1.07005825 -0.67888863
   0.94658104  2.07801688  2.31592941 -0.21670295 -0.55755989  1.7405184
   1.05395763  0.07194224 -0.69649989]
 [ 0.75975286  0.86338712  1.4507302   0.14450187  0.2609341  -0.67888863
  -0.34944544 -0.48122804  2.31592941 -0.21670295 -0.55755989 -0.30495667
  -1.01402715  0.07194224  0.74423876]
 [ 0.75975286  0.80293469  1.4507302   2.15207357  1.59192645 -0.67888863
  -0.34944544 -0.48122804 -0.56836639 -0.21670295  1.63159929 -0.30495667
   1.05395763  0.07194224  0.74423876]]


In [23]:
k_values = list(range(1, 31)) # testando valores de 1 a 30

param_grid = {'n_neighbors': k_values}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_scaled, y_resampled)

best_k = grid_search.best_params_['n_neighbors']
print("Melhor valor de k encontrado:", best_k)

Melhor valor de k encontrado: 26


In [24]:
# Treine o modelo final com o melhor valor de k
final_model = KNeighborsClassifier(n_neighbors=best_k)
final_model.fit(X_train_scaled, y_resampled)

KNeighborsClassifier(n_neighbors=26)

In [25]:
# Faça previsões nos dados de teste
y_pred = final_model.predict(X_test_scaled)

# Imprima o relatório de classificação
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.70      0.77     22481
           1       0.79      0.91      0.84     27688

    accuracy                           0.82     50169
   macro avg       0.82      0.80      0.81     50169
weighted avg       0.82      0.82      0.81     50169



### Selecionando features com SelectKBests

In [26]:
selector = SelectKBest(score_func=f_classif, k=5)
X_train_selected = selector.fit_transform(X_resampled, y_resampled)
X_test_selected = selector.transform(X_test)

selected_columns = X_train.columns[selector.get_support()]
print("Características selecionadas:", selected_columns)

Características selecionadas: Index(['vlr_renda_media_fam', 'qtd_comodos_domic_fam', 'cod_material_piso_fam',
       'cod_abaste_agua_domic_fam', 'qtde_pessoas'],
      dtype='object')


In [27]:
k_values = list(range(1, 31)) # testando valores de 1 a 30

param_grid = {'n_neighbors': k_values}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_selected, y_resampled)

best_k = grid_search.best_params_['n_neighbors']
print("Melhor valor de k encontrado:", best_k)

Melhor valor de k encontrado: 29


In [28]:
# Treine o modelo final com o melhor valor de k
final_model = KNeighborsClassifier(n_neighbors=best_k)
final_model.fit(X_train_selected, y_resampled)

KNeighborsClassifier(n_neighbors=29)

In [29]:
# Faça previsões nos dados de teste
y_pred_selected = final_model.predict(X_test_selected)

# Imprima o relatório de classificação
print(classification_report(y_test, y_pred_selected))

              precision    recall  f1-score   support

           0       0.81      0.82      0.82     22481
           1       0.85      0.84      0.85     27688

    accuracy                           0.83     50169
   macro avg       0.83      0.83      0.83     50169
weighted avg       0.83      0.83      0.83     50169

